# Exercise 2: Swendsen-Wang algorithm for the 2D Ising model

The goal of this exercise is to implement the Swendsen-Wang algorithm for the 2D Ising
model. Recall that the update of the Swendsen-Wang algorithm works as follows:

- Assign to each bond $b$ a variable $w_b \in \{ 0, 1 \}$, where 1 means “connected” and 0
means “disconnected”. If the spins connected by the bond $b = (n, m)$ between index
$n$ and $m$ are anti-parallel, always choose $w_b = 0$, If the spins are parallel, choose
$w_b = 0$ with probability $e^{−2\Beta J} , where $\Beta = k_bT$. In terms of conditional probabilities:

\begin{align}
p(w_b = 0 | \sigma_n \neq \sigma_m) &= 1 \\
p(w_b = 0 | \sigma_n = \sigma_m) &= e^{−2\Beta J} \\
p(w_b = 1 | \sigma_n \neq \sigma_m) &= 0 \\
p(w_b = 1 | \sigma_n = \sigma_m) &= 1 − e^{−2\Beta J} \\
\end{align}

- Interpreting the connected bonds as edges of a graph (where 0 means no edge) and
the spins as node, find “clusters” of spins, i.e., connected components of the graph.
- Flip each cluster (= connected component) with probability $p = 0.5$. Notice that a
single spin is also a “cluster”.
After the update, do a measurement (if the system is already thermalized) and continue
with the next update